In [3]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# trials segmentation notebook to trim the videos and update the datasets based on the trial start and trial ends times in the csv

In [ ]:
import pandas as pd
import numpy as np
import os
import imageio_ffmpeg
import subprocess as sp

In [3]:
base_path = r"C:\Users\shahd\Box\Awake Project\Maze data\simplermaze\mouse 6357\raw_cross_check"


session_paths= []

for i in os.listdir(base_path):
    if ".csv" in i:
        session_path = os.path.join(base_path, i)
        session_paths.append(session_path)

sessions_ID = ["3.6", "3.7", "3.8"]

sessions_path_dict = dict(zip(sessions_ID, session_paths))

# # for i, j in sessions.items():
# #     print (i, j)


print(sessions_path_dict)


{'3.6': 'C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\raw_cross_check\\session3.6.csv', '3.7': 'C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\raw_cross_check\\session3.7.csv', '3.8': 'C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\raw_cross_check\\session3.8.csv'}


In [ ]:
def make_csvs_no_empty_rows(session_dict):

    for i, j in session_dict.items():
        print(j)

        miao = pd.read_excel(j)
        miao = miao.dropna(how="all")

        miao.to_csv(base_path + f"/session{i}.csv", index = False)


# make_csvs_no_empty_rows(sessions_path_dict)

C:\Users\aleja\Box\Awake Project\Maze data\simplermaze\mouse 6357\raw_cross_check\mouse6357session3.6_data_2024-08-28_11_58_14.xlsx
C:\Users\aleja\Box\Awake Project\Maze data\simplermaze\mouse 6357\raw_cross_check\mouse6357session3.7_data_2024-08-29_10_23_02.xlsx
C:\Users\aleja\Box\Awake Project\Maze data\simplermaze\mouse 6357\raw_cross_check\mouse6357session3.8_data_2024-08-30_10_07_55.xlsx


In [ ]:
mouse_6357_path = r"C:\Users\shahd\Box\Awake Project\Maze data\simplermaze\mouse 6357"

exp_session_paths = []
for exp_session in os.listdir(mouse_6357_path):
    for i in sessions_ID:
        if i in exp_session:
            # print (exp_session)
            exp_session_path = os.path.join(mouse_6357_path, exp_session)
            exp_session_paths.append(exp_session_path)

print(exp_session_paths)

['C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-28_11_58_146357session3.6', 'C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-29_10_23_026357session3.7', 'C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-30_10_07_556357session3.8']


In [18]:
def trim_video_into_segments(parent_path, trials_csv_path, session):

    

    ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()
    #make output directory
    output_dir = os.path.join(parent_path, "new_segments\\")
    os.makedirs(output_dir, exist_ok=True)

    # look for the .mp4 movie in the parent path

    for i in os.listdir(parent_path):
        if ".mp4" in i:
            video = os.path.join(parent_path, i)

    trial_df = pd.read_csv(trials_csv_path)

    trial_df["trial_ID"]= [f"trial_{i:03}".format(i) for i in range(len(trial_df))]
    trial_df["state_sequence"] = [0 for i in range(len(trial_df)) ]
    trial_df["video_path"] = [f"{output_dir}trial_{i:03}.mp4".format(i) for i in range(len(trial_df))]
    trial_df["trial_start_time_s"]= (trial_df["trial_start_time"] / 1000)
    trial_df["end_trial_time_s"] = (trial_df["end_trial_time"]/ 1000)
    trial_df["duration_s"] = trial_df["end_trial_time_s"] - trial_df["trial_start_time_s"]
    trial_df["start_trial_frame"] = round(trial_df["trial_start_time_s"]*30)
    trial_df["end_trial_frame"] = round(trial_df["end_trial_time_s"]*30)

    
    trial_df.to_csv(parent_path + f"/new_session{session}_trials.csv", index = False)
    print("saved df to csv")

    # trial_df.head(12)

    for index, row in trial_df.iterrows():
        # print(index)

        start_time = row["trial_start_time_s"]  # Start at 0 seconds
        end_time = row["end_trial_time_s"]    # End at 7 seconds
        target_path = row["video_path"]
        
        print(f" trial{index}\nExtracting from {start_time:.2f}s to {end_time:.2f}s")
        # ffmpeg_extract_subclip(video, start_time, end_time, target_path)

        sp.call([ffmpeg_path, '-loglevel', 'quiet', '-ss', str(start_time), '-to', str(end_time), '-i', video, '-c', 'copy', f"{target_path}"])


In [ ]:
print(exp_session_paths)
print(sessions_path_dict)
print(sessions_ID)

for session in sessions_ID:
    for parent_path in exp_session_paths:
        if session in parent_path:
            trials_csv_path =  sessions_path_dict[session]
            print(f"session: {session}, parent path: {parent_path}, trials_csv_path: {trials_csv_path}")
            # trim_video_into_segments(parent_path, trials_csv_path, session)

['C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-28_11_58_146357session3.6', 'C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-29_10_23_026357session3.7', 'C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-30_10_07_556357session3.8']
{'3.6': 'C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\raw_cross_check\\session3.6.csv', '3.7': 'C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\raw_cross_check\\session3.7.csv', '3.8': 'C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\raw_cross_check\\session3.8.csv'}
['3.6', '3.7', '3.8']
session: 3.6, parent path: C:\Users\shahd\Box\Awake Project\Maze data\simplermaze\mouse 6357\2024-08-28_11_58_146357session3.6, trials_csv_path: C:\Users\shahd\Box\Awake Project\Maze data\simplermaze\mouse 6357\raw_cross_check\session3.6.csv
saved df to csv
 trial0
Extracting from 0.00s to 84.7